In [1]:
import numpy as	np
from tqdm import tqdm

In [24]:
input_file = '../data/Multibias/wiki.de.vec'
embeddings_file = input_file[:-4] + '.wv.npy'
vocabulary_file = input_file[:-4] + '.vocab'
embeddings_file, vocabulary_file

('../data/Multibias/wiki.de.wv.npy', '../data/Multibias/wiki.de.vocab')

In [25]:
import mmap

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines
num_lines = get_num_lines(input_file)

In [26]:
words = []
vectors = []
with open(input_file, 'r') as f:
    for i,line in enumerate(tqdm(f,total=num_lines)):
        fields = line.rstrip().split(' ')
        if len(fields) == 2:
            print(fields)
            continue
        try:
            word = fields[0]
            vector = np.fromiter((float(x) for x in fields[1:]),dtype=np.float)
            words.append(word)
            vectors.append(vector)
        except:
            print(fields)

matrix = np.array(vectors)
np.save(embeddings_file, matrix)
text = '\n'.join(words)
with open(vocabulary_file, 'w') as f:
    f.write(text)
del words, vectors

  0%|          | 1010/2275234 [00:00<07:55, 4782.20it/s]

['2275233', '300']


100%|██████████| 2275234/2275234 [07:21<00:00, 5155.07it/s]


# Translation

In [13]:
from googletrans import Translator
import pandas as pd
import json

translator = Translator()

def fun(inp):
    global src
    global target
    result = translator.translate(inp, dest=target, src=src)
    return result.text

In [9]:
df = pd.read_csv("../data/lists/female_word_file.txt",header=None)

src = 'en'
target_langs = ['es','de','fr']
for lang in target_langs:
    target = lang
    df[lang] = df[0].apply(fun)
    print(f'{lang} done')

df.to_csv("../data/lists/female_word_file.csv", index=False)

print('done')

es done
de done
fr done
done


In [10]:
df = pd.read_csv("../data/lists/female_word_file.csv")
es = list(df['es'])
fr = list(df['fr'])
de = list(df['de'])
es = [i.lower() for i in es]
fr = [i.lower() for i in fr]
de = [i.lower() for i in de]
with open('../data/lists/female_word_file_es.txt','w') as f:
    f.write("\n".join(es))
with open('../data/lists/female_word_file_fr.txt','w') as f:
    f.write("\n".join(fr))
with open('../data/lists/female_word_file_de.txt','w') as f:
    f.write("\n".join(de))


In [14]:
with open('../../debiaswe/data/gender_specific_full.json') as f:
    gendered_words = json.load(f)

src = 'en'
target_langs = ['es','de','fr']
gendered = {}
for lang in target_langs:
    gendered[lang] = []
    target = lang
    for i in gendered_words:
        gendered[lang].append(fun(i.replace('_',' ')).lower().replace(' ','_'))
    print(f'{lang} done')
with open('../../debiaswe/data/gender_specific_full_multi_lang.json','w') as f:
    json.dump(gendered,f,indent=4)

In [36]:
with open('../../debiaswe/data/equalize_pairs.json') as f:
    definitional_pairs = json.load(f)

src = 'en'
target_langs = ['es','de','fr']
definitional = {}
for lang in target_langs:
    definitional[lang] = []
    target = lang
    for i in definitional_pairs:
        a = i[0].replace('_',' ').lower()
        b = i[1].replace('_',' ').lower()
        definitional[lang].append([fun(a).replace(' ','_'), fun(b).replace(' ','_')])
    print(f'{lang} done')
with open('../../debiaswe/data/equalize_pairs_multi_lang.json','w') as f:
    json.dump(definitional,f,indent=4)

es done
de done
fr done


In [10]:
import json
with open('/mnt/c/Stanford/debiaswe/data/gender_specific_full_multi_lang.json') as f:
    data = json.load(f)

In [13]:
with open('/mnt/c/Stanford/debiaswe/data/gender_specific_full_fr.json','w') as f:
    json.dump(data['fr'],f,indent=4)